In [1]:
import numpy as np
import pandas as pd
from scipy import optimize
from tensorflow import keras
from tensorflow.compat.v1 import disable_eager_execution
disable_eager_execution()

# 读取数据
jia_data = pd.read_excel('雷达数据.xlsx', sheet_name='第一组飞机甲', header=0)
yi_data = pd.read_excel('雷达数据.xlsx', sheet_name='第二组飞机乙', header=0)
bing_data = pd.read_excel('雷达数据.xlsx', sheet_name='第三组飞机丙', header=0)

jia_data = jia_data.values / 10000
yi_data = yi_data.values / 10000
bing_data = bing_data.values / 10000


jia_y = jia_data[:, -1]
yi_y = (yi_data[:, -1] - np.average(yi_data[:, -1])) / 62 + np.average(yi_data[:, -1])
bing_y = (bing_data[:, -1] - np.average(bing_data[:, -1])) / 62 + np.average(bing_data[:, -1])

jia_x = jia_data[:, :3]
yi_x = yi_data[:, :3]
bing_x = bing_data[:, :3]

In [14]:

# 构建图模型，求解梯度，loss
def structure_graph(data_, y_):
    location = keras.backend.placeholder(shape=(3, ), dtype='float64',
                                         name='location')
    loss = keras.backend.variable(value=0.0, dtype='float64',
                                  name='square_loss')
    for i in range(len(y_)):
        loss = loss + keras.backend.square(keras.backend.square(location[0] - data_[i][0]) +
                                           keras.backend.square(location[1] - data_[i][1]) +
                                           keras.backend.square(location[2]) -
                                           np.square(y_[i]))

    grads = keras.backend.gradients(loss, location)[0]
    iterate_func = keras.backend.function(inputs=[location], outputs=[loss, grads])
    return iterate_func


func_jia = structure_graph(jia_x, jia_y)
func_yi = structure_graph(yi_x, yi_y)
func_bing = structure_graph(bing_x, bing_y)

In [15]:
class Evaluator:
    def __init__(self, func):
        self.loss_value = None
        self.grads_value = None
        self.iterate_func = func

    def get_loss(self, x_):
        outs = self.iterate_func(inputs=[x_])
        loss_value = outs[0]
        grads_value = outs[1]
        self.loss_value = loss_value
        self.grads_value = grads_value

        return loss_value

    def get_grads(self, x_):
        assert self.grads_value is not None
        grads_value = np.copy(self.grads_value)
        self.grads_value = None
        self.loss_value = None

        return grads_value


evaluator_jia = Evaluator(func_jia)
evaluator_yi = Evaluator(func_yi)
evaluator_bing = Evaluator(func_bing)


def intersection(P1, P2, P3, r1, r2, r3):
    temp1 = P2 - P1
    e_x = temp1 / np.linalg.norm(temp1)
    temp2 = P3 - P1
    i = np.dot(e_x, temp2)
    temp3 = temp2 - i * e_x
    e_y = temp3 / np.linalg.norm(temp3)
    e_z = np.cross(e_x, e_y)
    d = np.linalg.norm(P2 - P1)
    j = np.dot(e_y, temp2)
    x = (r1 * r1 - r2 * r2 + d * d) / (2 * d)
    y = (r1 * r1 - r3 * r3 - 2 * i * x + i * i + j * j) / (2 * j)
    temp4 = r1 * r1 - x * x - y * y
    if temp4 < 0:
        raise Exception("三球体无交点！")
    z = np.sqrt(temp4)
    p_12_a = P1 + x * e_x + y * e_y + z * e_z
    p_12_b = P1 + x * e_x + y * e_y - z * e_z
    if p_12_a[2] > 0:
        return p_12_a
    else:
        return p_12_b

In [16]:
x_jia = intersection(jia_x[4], jia_x[15], jia_x[28], jia_y[4], jia_y[15], jia_y[28])
x_yi = intersection(yi_x[1], yi_x[2], yi_x[27], yi_y[1], yi_y[2], yi_y[27])
x_bing = intersection(bing_x[2], bing_x[7], bing_x[10], bing_y[2], bing_y[7], bing_y[10])
print(x_jia)
print(x_yi)
print(x_bing)

def solve(evaluator_, x):

    # 设置迭代次数
    iteration = 20

    min_val = None
    for i in range(iteration):
         x, min_val, info = optimize.fmin_l_bfgs_b(evaluator_.get_loss,
                                                   x,
                                                   fprime=evaluator_.get_grads,
                                                   )

    print('共迭代{}次，损失基本不再减小，最终结果为：{}'.format(iteration, x))
    print('最终损失（loss）：{}'.format(min_val))
    evaluator_.get_loss(x)
    print('损失函数在该坐标点的梯度为：{}'.format(evaluator_.grads_value))


# 最终求解
print('\n甲组数据：')
solve(evaluator_jia, x_jia)

print('\n乙组数据：')
solve(evaluator_yi, x_yi)

print('\n丙组数据：')
solve(evaluator_bing, x_bing)


[1.1490028  0.03869765 1.30783873]
[0.67330788 0.28174126 4.64000561]
[ 0.8289271   0.53394719 10.4814536 ]

甲组数据：
共迭代20次，损失基本不再减小，最终结果为：[1.14216467 0.03433891 1.30933114]
最终损失（loss）：0.002373984732172337
损失函数在该坐标点的梯度为：[-1.26323194e-07  1.84576632e-07  6.66936944e-06]

乙组数据：
共迭代20次，损失基本不再减小，最终结果为：[0.71509468 0.26664807 4.63907698]
最终损失（loss）：0.0017852263122260102
损失函数在该坐标点的梯度为：[-8.43057138e-07 -6.07481588e-06 -1.65785452e-06]

丙组数据：
共迭代20次，损失基本不再减小，最终结果为：[ 0.92645928  0.49223834 10.48108106]
最终损失（loss）：0.003289288876495624
损失函数在该坐标点的梯度为：[ 6.11138247e-07  1.01983262e-06 -2.76107557e-07]
